In [28]:
import os
import cv2
import random
import numpy as np
from PIL import Image
from imageio import imwrite
from tqdm import tqdm
import click
import matplotlib.pyplot as plt
import tensorflow as tf
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [29]:
drone_folder = 'drone-20'
def compose(mask_path, bg_path, im_path, x_coordinate, y_coordinate, out_path, ground_truth_path, class_id, out_gt_path):
    bg_im = Image.open(bg_path)
    bg_im = np.array(bg_im)
    #mask = Image.open(mask_path)
    #mask = np.array(mask)
    mask = cv2.imread(mask_path, 0)
    #plt.imshow(mask)
    #plt.show()
    #kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (11, 11))
    #(thresh, binRed) = cv2.threshold(mask, 50, 255, cv2.THRESH_BINARY)
    #opening = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel, iterations=3)
    #mask = opening
    # mask = cv2.resize(mask, (mask.shape[0]//2,mask.shape[1]//2), interpolation = cv2.INTER_CUBIC)
    #plt.imshow(mask)
    #plt.show()
    #
    if len(mask.shape) == 2:
        mask = np.expand_dims(mask, -1)
    else:
        mask = np.expand_dims(np.mean(mask, axis=-1),-1)
    canvas = np.zeros((bg_im.shape[0], bg_im.shape[1], 1)).astype(np.uint8)
    #mask = cv2.copyMakeBorder(mask, (bg_im.shape[0]-mask.shape[0])//2, (bg_im.shape[0]-mask.shape[0])//2, 
    #(bg_im.shape[1]-mask.shape[1])//2, (bg_im.shape[1]-mask.shape[1])//2, cv2.BORDER_CONSTANT)
    canvas[0+(bg_im.shape[0]-mask.shape[0])//2: mask.shape[0]+(bg_im.shape[0]-mask.shape[0])//2,
     0+(bg_im.shape[1]-mask.shape[1])//2: mask.shape[1]+(bg_im.shape[1]-mask.shape[1])//2] = mask
    mask = canvas
    #cv2.imshow("image",canvas)
    #cv2.waitKey()
    req_x = y_coordinate
    req_y = x_coordinate
    x_min = 1500
    x_max = -1
    y_min = 1500
    y_max = -1
    for i in range(mask.shape[0]):
        for j in range(mask.shape[1]):
            if mask[i, j] > 150:
                mask[i,j] = 255
                if i < x_min:
                    x_min = i
                if i > x_max:
                    x_max = i
                if j < y_min:
                    y_min = j
                if j > y_max:
                    y_max = j

    centre_x = (x_min + x_max) // 2
    centre_y = (y_min + y_max) // 2
    delta_x = req_x - centre_x
    delta_y = req_y - centre_y
    T = np.float32([[1, 0, delta_y], [0, 1, delta_x]])
    translated_mask = cv2.warpAffine(np.float32(mask), T, (mask.shape[1], mask.shape[0]))
    translated_mask = translated_mask.astype(np.uint8)
    #plt.imshow(translated_mask)
    #plt.show()
    fg = Image.open(im_path)
    fg = np.array(fg)
    #print(fg.shape)
    #fg = cv2.copyMakeBorder(fg, (bg_im.shape[0]-fg.shape[0])//2, (bg_im.shape[0]-fg.shape[0])//2,
    # (bg_im.shape[1]-fg.shape[1])//2, (bg_im.shape[1]-fg.shape[1])//2, cv2.BORDER_CONSTANT)
    canvas = np.zeros(bg_im.shape).astype(np.uint8)
    canvas[0+(bg_im.shape[0]-fg.shape[0])//2: fg.shape[0]+(bg_im.shape[0]-fg.shape[0])//2,
     0+(bg_im.shape[1] - fg.shape[1])//2: fg.shape[1]+(bg_im.shape[1]-fg.shape[1])//2] = fg
    fg = canvas
    translated_fg = cv2.warpAffine(np.float32(fg), T, (fg.shape[1], fg.shape[0]))
    translated_fg = translated_fg.astype(np.uint8)
    
    composite_image = (bg_im * (1 - (np.tile(np.expand_dims(translated_mask, -1), 3) // 255))).astype(np.uint8) + ((np.tile(np.expand_dims(translated_mask, -1), 3) // 255) * translated_fg).astype(np.uint8)
    #plt.imshow(composite_image)
    #plt.show()
    x_min = 1500
    x_max = -1
    y_min = 1500
    y_max = -1
    for i in range(translated_mask.shape[0]):
        for j in range(translated_mask.shape[1]):
            if translated_mask[i, j] == 255:
                if i < x_min:
                    x_min = i
                if i > x_max:
                    x_max = i
                if j < y_min:
                    y_min = j
                if j > y_max:
                    y_max = j
    with open(ground_truth_path) as f:
        with open(os.path.basename(bg_path)[:-4] + '_' + os.path.basename(im_path)[:-4] + '.txt', "w") as f1:
            for line in f:
                f1.write(line)
    m_im = Image.open(mask_path)
    m_w,m_h = np.size(m_im)
    bg_im = Image.open(bg_path)
    bg_w,bg_h = np.size(bg_im)
    with open("/mnt/DATA/Abhay/Downloads/Abhay/drone_masks/"+drone_folder+"/cmp_annotations/"+os.path.basename(bg_path)[:-4] + '_' + os.path.basename(im_path)[:-4] + '.txt', 'a+') as file_object:
        file_object.write(class_id + " %f %f %f %f\n" % (x_coordinate/bg_w, y_coordinate/bg_h, m_w/bg_w, m_h/bg_h))
    imwrite("/mnt/DATA/Abhay/Downloads/Abhay/drone_masks/"+drone_folder+"/cmp_images/"+os.path.basename(bg_path)[:-4] + '_' + os.path.basename(im_path), composite_image)
    #print(os.path.basename(bg_path)[:-4]+'.txt')
#if __name__ == '__main__':
    #compose(mask_path = "/mnt/DATA/Abhay/Downloads/Abhay/test_mask1.jpg", bg_path = "/mnt/DATA/Abhay/Downloads/Abhay/V_DRONE_0013_009.png", im_path = "/mnt/DATA/Abhay/Downloads/Abhay/test_track1.jpg", x_coordinate = 300, y_coordinate=300, out_path="nw.jpg", ground_truth_path = "temp.txt", class_id="0", out_gt_path="test_00000216.jpg")


In [30]:
m_path = "/mnt/DATA/Abhay/Downloads/Abhay/test_mask1.jpg"
m_im = Image.open(m_path)
m_w,m_h = np.size(m_im)
print(m_w,m_h)

bg_path = "/mnt/DATA/Abhay/Downloads/Abhay/V_DRONE_0013_009.png"
bg_im = Image.open(bg_path)
bg_w,bg_h = np.size(bg_im)
print(bg_w,bg_h)
os.path.basename(m_path)[:10]

92 91
640 512


'test_mask1'

#### Foreground Image Mask paths / mask_path 

In [32]:
m_path = "/mnt/DATA/Abhay/Downloads/tf-siammask/drone_masks/"+drone_folder+"/crop_mask"
mask_list = os.listdir(m_path)
mask_path_list = []

for i in range(len(mask_list)):
    path = m_path + "/" + mask_list[i]
    mask_path_list.append(path)
print((mask_path_list[0]),mask_list[0])
len(mask_path_list)

/mnt/DATA/Abhay/Downloads/tf-siammask/drone_masks/drone-20/crop_mask/drone-20_00000204.jpg drone-20_00000204.jpg


2316

### Image/Track Path

In [34]:
im_path = "/mnt/DATA/Abhay/Downloads/tf-siammask/drone_masks/"+drone_folder+"/crop_img"
im_list = os.listdir(im_path)
im_path_list = []

for i in range(len(im_list)):
    path = im_path +'/'+im_list[i]
    im_path_list.append(path)
#im_path_list.remove("/mnt/DATA/Abhay/Downloads/tf-siammask/drone/drone-14/img/00000001.jpg")
#im_path_list.remove("/mnt/DATA/Abhay/Downloads/tf-siammask/drone/drone-14/img/00001735.jpg")

print((im_path_list[0]))
print(mask_path_list[0])
print(len(im_path_list))

/mnt/DATA/Abhay/Downloads/tf-siammask/drone_masks/drone-20/crop_img/drone-20_00000204.jpg
/mnt/DATA/Abhay/Downloads/tf-siammask/drone_masks/drone-20/crop_mask/drone-20_00000204.jpg
2316


### Center Coordinates

In [35]:
center_path = "/mnt/DATA/Abhay/Downloads/Abhay/DroneTrackingDataset/new_centres_0.0009.txt"

with open(center_path) as f:
    lines = f.readlines()
new_list = []
bg_path = "/mnt/DATA/Abhay/Downloads/Abhay/DroneTrackingDataset/Drone_Snippet"
for i in range(0,len(mask_path_list)):
    a = lines[i].split(" ")
    new_list.append(a)
    new_list[i][0] = bg_path+"/"+new_list[i][0]


print((new_list[0][0]))
print((new_list[0][0][67:]))
print((new_list[0][0][67:83]))
print(len(new_list))
# bg_path = "/mnt/DATA/Abhay/Downloads/Abhay/dataset_Base_And_SynData/Drone_SD_cl2/images/train"
# bg_list = os.listdir(bg_path)
# bg_path_list=[]

# for i in range(len(bg_list)):
#     path = bg_path+"/"+bg_list[i]
#     bg_path_list.append(path)
# print((bg_path_list[0:10]))
x = int(float(new_list[i][1]))
y = int(float(new_list[i][2]))


/mnt/DATA/Abhay/Downloads/Abhay/DroneTrackingDataset/Drone_Snippet/V_DRONE_0974_275.png
V_DRONE_0974_275.png
V_DRONE_0974_275
2316


In [ ]:
for i in range(len(mask_list)):
    m_path = mask_path_list[i]
    i_path = im_path_list[i]
    b_path = new_list[i][0]
    x = int(float(new_list[i][1]))
    y = int(float(new_list[i][2]))
    o_path = "/mnt/DATA/Abhay/Downloads/tf-siammask/drone/"+drone_folder+"/com_images/" + im_list[i]
    gt_path = "temp.txt"
    o_gt_path = o_path
    compose(mask_path=m_path,bg_path=b_path, im_path=i_path, x_coordinate=x,y_coordinate=y,out_path=o_path,ground_truth_path=gt_path,class_id="0", out_gt_path=o_path)    
    comp_ann_path = "/mnt/DATA/Abhay/Downloads/Abhay/Composed_files/annotations/"+os.path.basename(b_path)[:-4] + '_' + os.path.basename(i_path)[:-4] + '.txt'
    org_ann_path = '/mnt/DATA/Abhay/Downloads/Abhay/DroneTrackingDataset/Drone_TXTs_100Snippet/' + os.path.basename(b_path)[:-4] + '.txt'

    f = open('/mnt/DATA/Abhay/Downloads/Abhay/DroneTrackingDataset/Drone_TXTs_100Snippet/' + os.path.basename(b_path)[:-4] + '.txt','r')
    x = f.readlines()
    #print(x)

    f1 = open("/mnt/DATA/Abhay/Downloads/Abhay/drone_masks/"+drone_folder+"/cmp_annotations/"+os.path.basename(b_path)[:-4] + '_' + os.path.basename(i_path)[:-4] + '.txt','a')
    x1 = f1.writelines(x)
    f1.close()

    f2 = open("/mnt/DATA/Abhay/Downloads/Abhay/drone_masks/"+drone_folder+"/cmp_annotations/"+os.path.basename(b_path)[:-4] + '_' + os.path.basename(i_path)[:-4] + '.txt','r')
    x2 = f2.readlines()
    print(x2)
    f2.close()
    